##  리카르도 비교우위 모형을 강화학습으로 실험해보자 <br>(3개이상의 국가, 3개이상의 물품으로)

|  <center></center> |  <center>미국</center> |  <center>일본</center> | <center>중국</center> |
|:--------|:--------:|--------:|
|**신발** | <center>800 </center> |<center>1000</center> |<center>2000</center> |
|**철** | <center>1000 </center> |<center>1000</center> |<center>800</center> |
|**휴대폰** | <center>1200 </center> |<center>700</center> |<center>1300</center> |

위의 표와 같은 교역조건이 주어졌을 때 각 나라들은 어떤 물건을 생산하는 것이 좋을까요?

핵심 아이디어는
1. 무역이 일어나면 서로간에 이득이다. -> 무역이 성사되면 reward가 올라간다.
2. 무역이 일어나기 위해서는 수요자와 공급자의 가격이 맞아야하고, 다른 경쟁나라보다 가격우위에 있어야 합니다.

이와 같은 아이디어를 가지고, bandit, epsilon-greedy 방법으로 무역이 어떻게 진행될지 알아보았습니다


In [1]:
import numpy as np

In [2]:
class agent:
    def __init__(self,name, n_variety,shoes_q,steel_q,phone_q):
        self.name=name
        self.shoes_p = [_, steel_q/shoes_q, phone_q/shoes_q]
        self.steel_p = [shoes_q/steel_q, _,  phone_q/steel_q] 
        self.phone_p = [shoes_q/phone_q, steel_q/phone_q, _]
        
        self.price_plan = [self.shoes_p,self.steel_p, self.phone_p]
        self.value=np.zeros(n_variety)
        self.commo= np.random.randint(3)
    
    def get_reward(self,):
        self.value[self.commo]+=1
            
    def choose_commodity(self, epsilon):
        rand_num=np.random.random()
        if epsilon>rand_num:
            return np.random.randint(3)
        else:
            return np.argmax(self.value)    
        

In [3]:
import time
from collections import defaultdict

In [4]:
product=['shoes','steel','phone']
result=dict()
def trade(agent1 ,agent2 , competitor , epsilon):
    commo1 = agent1.choose_commodity(epsilon)
    commo2 = agent2.choose_commodity(epsilon)
    
    
    #같은 물건은 무역하지 않음
    if commo1 == commo2 :
        return False
    
    (print("trade process started between {agent1} , {agent2}, {commo1},{commo2}".format(agent1=agent1.name,agent2=agent2.name, 
                                                                                         commo1=commo1,commo2=commo2)))
    flag=True
    #각자 자기가진 물건보다 상대의 것이 낫다고 판단되면 교환, 기준은 상대가격
    if agent1.price_plan[commo1][commo2]< agent1.price_plan[commo2][commo1] and \
    agent2.price_plan[commo2][commo1] < agent2.price_plan[commo1][commo2]:
        flag=True

    superior_commo1= competitor.price_plan[commo1][commo2]< agent1.price_plan[commo1][commo2]
    if superior_commo1 :
        agent1.value[commo1]-=0.5
        competitor.value[commo1]+=1
        print("{competitor} interrupted {agent1}, goods={goods}".format(competitor= competitor.name, agent1 = agent1.name, goods=commo1))
        
    superior_commo2= competitor.price_plan[commo2][commo1]< agent2.price_plan[commo2][commo1]
    if superior_commo2 :
        agent2.value[commo2]-=0.5
        competitor.value[commo2]+=1
        print("{competitor} interrupted {agent2}, goods={goods}".format(competitor= competitor.name, agent2 = agent2.name, goods=commo2))
        
    if superior_commo1 or superior_commo2 :
        flag=False
    
    if flag:
        agent1.value[commo1]+=1
        agent2.value[commo2]+=1
        print("trade succeded between {0},{1}, goods : {commo1},{commo2}".format(agent1.name,agent2.name, commo1 = commo1, commo2 =commo2 ))
        
        trade_set=(product[commo1],product[commo2])
        if trade_set not in result:
            result[trade_set]=[(agent1.name,agent2.name),1]
        else:
            result[trade_set][1]+=1
#     time.sleep(1)
    return flag

In [5]:
from itertools import combinations

In [6]:
n_variety=3
USA=agent("USA",n_variety,800,1000,1200)
JAPAN=agent("JAPAN",n_variety,1000,1000,700)
CHINA=agent("CHINA",n_variety,2000,800,1300)
pairs=[[USA,JAPAN,CHINA],[USA,CHINA,JAPAN],[JAPAN,CHINA,USA]]

epsilon=0.1


In [7]:
def experiment(N_episode,epsilon):
    history = []
    for i in range(N_episode):
        
        for pair in pairs:
            trade(*pair,epsilon)

experiment(500,0.3)

trade process started between USA , JAPAN, 0,2
CHINA interrupted USA, goods=0
trade process started between USA , CHINA, 1,0
trade succeded between USA,CHINA, goods : 1,0
trade process started between USA , JAPAN, 1,0
CHINA interrupted JAPAN, goods=0
trade process started between USA , CHINA, 1,0
trade succeded between USA,CHINA, goods : 1,0
trade process started between USA , CHINA, 1,0
trade succeded between USA,CHINA, goods : 1,0
trade process started between JAPAN , CHINA, 1,2
trade succeded between JAPAN,CHINA, goods : 1,2
trade process started between USA , CHINA, 1,0
trade succeded between USA,CHINA, goods : 1,0
trade process started between USA , CHINA, 1,0
trade succeded between USA,CHINA, goods : 1,0
trade process started between USA , CHINA, 1,0
trade succeded between USA,CHINA, goods : 1,0
trade process started between USA , JAPAN, 1,0
CHINA interrupted JAPAN, goods=0
trade process started between USA , JAPAN, 1,0
CHINA interrupted JAPAN, goods=0
trade process started betwe

USA interrupted JAPAN, goods=1
trade process started between USA , JAPAN, 1,0
CHINA interrupted JAPAN, goods=0
trade process started between USA , CHINA, 1,0
trade succeded between USA,CHINA, goods : 1,0
trade process started between JAPAN , CHINA, 2,0
USA interrupted JAPAN, goods=2
trade process started between USA , CHINA, 1,2
JAPAN interrupted USA, goods=1
trade process started between JAPAN , CHINA, 1,0
USA interrupted JAPAN, goods=1
trade process started between JAPAN , CHINA, 2,0
USA interrupted JAPAN, goods=2
trade process started between USA , JAPAN, 1,0
CHINA interrupted JAPAN, goods=0
trade process started between USA , CHINA, 1,0
trade succeded between USA,CHINA, goods : 1,0
trade process started between JAPAN , CHINA, 1,0
USA interrupted JAPAN, goods=1
trade process started between USA , JAPAN, 1,2
CHINA interrupted JAPAN, goods=2
trade process started between USA , CHINA, 1,0
trade succeded between USA,CHINA, goods : 1,0
trade process started between JAPAN , CHINA, 2,1
USA

In [8]:
USA.value

array([-25.5, 593. ,  96. ])

In [9]:
JAPAN.value


array([-46. , -44.5, -47. ])

In [10]:
CHINA.value

array([525., -12., 122.])

In [11]:
result

{('phone', 'shoes'): [('USA', 'CHINA'), 48],
 ('steel', 'phone'): [('JAPAN', 'CHINA'), 47],
 ('steel', 'shoes'): [('USA', 'CHINA'), 323]}

## Insight 1 : 
JAPAN 은 생산할 수 있는 물건 중에 다른 물품으로 나타낸 상대가격에서 경쟁자를 누를 수 있는 비교우위 상품이 없음. <br>따라서, 무역을 못함
->epsilon 을 0.1로만 실험했을 때의 오해인줄 알았으나, epsilon 높여도 같은 결과 -> 인줄 알았으나 철-휴대폰시장에서 일본은 중국과 무역할 수 있음


## Insight2 :
USA는 철에 CHINA는 신발과 핸드폰 생산에 비교우위가 있음


## Insight3 :
epsion 을 0.1로 놓느냐, 0.3으로 놓느냐에 따라서도 value 값이 달라짐